<a href="https://colab.research.google.com/github/metdinov/statstests/blob/master/Statistical_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import altair as alt
import numpy as np
import pandas as pd
import scipy.stats as stats

np.random.seed(42)

In [0]:
def generate_sample(dist, all_params, size):
  distributions = {"Normal": stats.norm, 
                 "Exponential": stats.expon,
                 "Beta": stats.beta,
                 "Student t": stats.t,
                 "Generalized Pareto": stats.genpareto,
                 "Gamma": stats.gamma,
                 "Erlang": stats.erlang}
  
  params = []
  if dist == "Student t":
    params.append(all_params["df"])
  elif dist == "Beta":
    params.append(all_params["alpha"])
    params.append(all_params["beta"])
  elif dist == "Generalized Pareto":
    params.append(all_params["shape"])
  elif dist == "Gamma" or dist == "Erlang":
    params.append(all_params["alpha"])
    
  params.append(all_params["mean"])
  params.append(all_params["scale"])
  
  return distributions[dist].rvs(*params, size=size)
    

In [0]:
#@title Distributions to compare { run: "auto", display-mode: "both" }
dist1 = "Gamma" #@param ["Normal", "Exponential", "Beta", "Student t", "Generalized Pareto", "Gamma", "Erlang"]

# Set distribution parameters
mean1 = 0 #@param {type:"number"}
scale1 = 1 #@param {type:"number"}
params1 = {"mean": mean1,
           "scale": scale1,
           "df": 50, # Degrees of freedom (Student t)
           "alpha": 1,
           "beta": 2,
           "shape": 20}


dist2 = "Student t" #@param ["Normal", "Exponential", "Beta", "Student t", "Generalized Pareto", "Gamma", "Erlang"]
mean2 = 0 #@param {type:"number"}
scale2 = 1 #@param {type:"number"}
params2 = {"mean": mean2,
           "scale": scale2,
           "df": 50, # Degrees of freedom (Student t)
           "alpha": 1,
           "beta": 2,
           "shape": 3}


#@markdown ### Sample size
size = 500 #@param {type:"integer"}

sample1 = generate_sample(dist1, params1, size)
sample2 = generate_sample(dist2, params2, size)

data = pd.DataFrame({"sample1": sample1,
                     "sample2": sample2})


In [24]:
data.describe()

,sample1,sample2
count,500.000000,500.000000
mean,1.001462,-0.059639
std,1.040444,0.996707
min,0.003937,-3.088195
25%,0.292304,-0.764148
50%,0.634678,-0.080207
75%,1.418849,0.602935
max,7.315908,2.677033


In [25]:
# Plot histograms
hist1 = alt.Chart(data).mark_bar().encode(
            alt.X("sample1", bin=True),
            y='count()',
)

hist2 = alt.Chart(data).mark_bar().encode(
            alt.X("sample2", bin=True),
            y='count()',
)

hist1 | hist2

HConcatChart({
  hconcat: [Chart({
    data:       sample1   sample2
    0    1.911991  0.175216
    1    0.806930 -1.107229
    2    1.265380  0.618771
    3    2.273400  0.035307
    4    2.320155  0.883242
    5    0.398255 -0.807189
    6    3.046000 -1.030739
    7    0.025191 -0.419996
    8    0.472945 -1.211192
    9    2.261432  0.941445
    10   0.321083  1.126178
    11   0.318941  1.690104
    12   0.360428 -0.097426
    13   0.947558 -0.073628
    14   0.576846 -1.068781
    15   1.297313 -0.611069
    16   2.252944 -0.317842
    17   1.121470  0.664048
    18   0.228743  1.173061
    19   2.698326  2.060071
    20   0.307195  0.882682
    21   0.162395 -0.575476
    22   0.094275  0.020956
    23   0.828701 -0.874822
    24   0.120770 -0.172696
    25   0.149401  0.915929
    26   0.356718 -0.321690
    27   0.330803  1.968462
    28   0.926946  0.471685
    29   1.696878  0.194657
    ..        ...       ...
    470  0.373103 -0.235127
    471  0.129063  0.267757
    472  1.999959  1.403297
    473  0.680121 -1.082342
    474  0.848975 -0.657296
    475  0.842652  2.588746
    476  0.528469 -1.799823
    477  0.213849  0.599596
    478  1.367012 -0.030218
    479  0.135884  1.958427
    480  0.109674  0.227459
    481  1.238375  0.256394
    482  0.234251 -1.622424
    483  0.366397 -0.815751
    484  0.644790  1.500998
    485  0.596658 -0.720551
    486  1.232987 -1.175771
    487  0.976656  0.690638
    488  1.033938 -2.050695
    489  1.251953  2.677033
    490  0.346177 -0.977905
    491  0.389100 -0.330760
    492  0.576249  0.172317
    493  0.367392 -1.799186
    494  4.195572  0.814804
    495  1.682288 -1.675837
    496  0.179910 -0.524650
    497  0.005610 -1.788092
    498  0.043725 -0.972653
    499  0.279798  0.157547
    
    [500 rows x 2 columns],
    encoding: EncodingWithFacet({
      x: X({
        bin: True,
        shorthand: 'sample1'
      }),
      y: Y({
        shorthand: 'count()'
      })
    }),
    mark: 'bar'
  }), Chart({
    data:       sample1   sample2
    0    1.911991  0.175216
    1    0.806930 -1.107229
    2    1.265380  0.618771
    3    2.273400  0.035307
    4    2.320155  0.883242
    5    0.398255 -0.807189
    6    3.046000 -1.030739
    7    0.025191 -0.419996
    8    0.472945 -1.211192
    9    2.261432  0.941445
    10   0.321083  1.126178
    11   0.318941  1.690104
    12   0.360428 -0.097426
    13   0.947558 -0.073628
    14   0.576846 -1.068781
    15   1.297313 -0.611069
    16   2.252944 -0.317842
    17   1.121470  0.664048
    18   0.228743  1.173061
    19   2.698326  2.060071
    20   0.307195  0.882682
    21   0.162395 -0.575476
    22   0.094275  0.020956
    23   0.828701 -0.874822
    24   0.120770 -0.172696
    25   0.149401  0.915929
    26   0.356718 -0.321690
    27   0.330803  1.968462
    28   0.926946  0.471685
    29   1.696878  0.194657
    ..        ...       ...
    470  0.373103 -0.235127
    471  0.129063  0.267757
    472  1.999959  1.403297
    473  0.680121 -1.082342
    474  0.848975 -0.657296
    475  0.842652  2.588746
    476  0.528469 -1.799823
    477  0.213849  0.599596
    478  1.367012 -0.030218
    479  0.135884  1.958427
    480  0.109674  0.227459
    481  1.238375  0.256394
    482  0.234251 -1.622424
    483  0.366397 -0.815751
    484  0.644790  1.500998
    485  0.596658 -0.720551
    486  1.232987 -1.175771
    487  0.976656  0.690638
    488  1.033938 -2.050695
    489  1.251953  2.677033
    490  0.346177 -0.977905
    491  0.389100 -0.330760
    492  0.576249  0.172317
    493  0.367392 -1.799186
    494  4.195572  0.814804
    495  1.682288 -1.675837
    496  0.179910 -0.524650
    497  0.005610 -1.788092
    498  0.043725 -0.972653
    499  0.279798  0.157547
    
    [500 rows x 2 columns],
    encoding: EncodingWithFacet({
      x: X({
        bin: True,
        shorthand: 'sample2'
      }),
      y: Y({
        shorthand: 'count()'
      })
    }),
    mark: 'bar'
  })]
})

In [26]:
# Student's t test
t_value, p_value = stats.ttest_ind(sample1, sample2)
xs_student = np.linspace(stats.t.ppf(0.001, size-1), stats.t.ppf(0.999, size-1), 200)
tests = pd.DataFrame({"t_value": xs_student, "density": stats.t.pdf(xs_student, 2*size-2)})

chart = alt.Chart(tests, title="Student's t Distribution").mark_line(opacity=0.3).encode(alt.X("t_value"), alt.Y("density", scale=alt.Scale(domain=[0, 0.45])))

point_df = pd.DataFrame({"t_value": [t_value], "density": [stats.t.pdf(t_value, size-1)], "p_value": [p_value]})
point = alt.Chart(point_df).mark_circle(color="red", size=60).encode(x="t_value", y="density", tooltip="p_value")
chart + point

LayerChart({
  layer: [Chart({
    data:       density   t_value
    0    0.003259 -3.106645
    1    0.003586 -3.075422
    2    0.003942 -3.044199
    3    0.004330 -3.012977
    4    0.004751 -2.981754
    5    0.005208 -2.950532
    6    0.005704 -2.919309
    7    0.006241 -2.888087
    8    0.006823 -2.856864
    9    0.007451 -2.825642
    10   0.008129 -2.794419
    11   0.008861 -2.763196
    12   0.009649 -2.731974
    13   0.010498 -2.700751
    14   0.011410 -2.669529
    15   0.012389 -2.638306
    16   0.013440 -2.607084
    17   0.014566 -2.575861
    18   0.015771 -2.544639
    19   0.017060 -2.513416
    20   0.018436 -2.482193
    21   0.019904 -2.450971
    22   0.021469 -2.419748
    23   0.023134 -2.388526
    24   0.024905 -2.357303
    25   0.026785 -2.326081
    26   0.028780 -2.294858
    27   0.030893 -2.263636
    28   0.033130 -2.232413
    29   0.035495 -2.201190
    ..        ...       ...
    170  0.035495  2.201190
    171  0.033130  2.232413
    172  0.030893  2.263636
    173  0.028780  2.294858
    174  0.026785  2.326081
    175  0.024905  2.357303
    176  0.023134  2.388526
    177  0.021469  2.419748
    178  0.019904  2.450971
    179  0.018436  2.482193
    180  0.017060  2.513416
    181  0.015771  2.544639
    182  0.014566  2.575861
    183  0.013440  2.607084
    184  0.012389  2.638306
    185  0.011410  2.669529
    186  0.010498  2.700751
    187  0.009649  2.731974
    188  0.008861  2.763196
    189  0.008129  2.794419
    190  0.007451  2.825642
    191  0.006823  2.856864
    192  0.006241  2.888087
    193  0.005704  2.919309
    194  0.005208  2.950532
    195  0.004751  2.981754
    196  0.004330  3.012977
    197  0.003942  3.044199
    198  0.003586  3.075422
    199  0.003259  3.106645
    
    [200 rows x 2 columns],
    encoding: EncodingWithFacet({
      x: X({
        shorthand: 't_value'
      }),
      y: Y({
        scale: Scale({
          domain: [0, 0.45]
        }),
        shorthand: 'density'
      })
    }),
    mark: MarkDef({
      opacity: 0.3,
      type: 'line'
    }),
    title: "Student's t Distribution"
  }), Chart({
    data:         density       p_value   t_value
    0  2.998834e-48  4.394150e-54  16.46772,
    encoding: EncodingWithFacet({
      tooltip: Tooltip({
        shorthand: 'p_value'
      }),
      x: X({
        shorthand: 't_value'
      }),
      y: Y({
        shorthand: 'density'
      })
    }),
    mark: MarkDef({
      color: 'red',
      size: 60,
      type: 'circle'
    })
  })]
})

In [27]:
# Mann-Whitney's U (similar to Wilcoxon but for independent samples)
statistic, p_value = stats.mannwhitneyu(sample1, sample2)
print(statistic)
print(p_value)

55012.0
2.569613135740974e-53


In [0]:
# Brunner-Munzel in dev stage in scipy
# https://scipy.github.io/devdocs/generated/scipy.stats.brunnermunzel.html

from collections import namedtuple
 
BrunnerMunzelResult = namedtuple('BrunnerMunzelResult', ('statistic','pvalue'))
 
def brunner_munzel_test(x, y, alternative="two_sided", alpha=0.5):
     
    """
    Computes the Brunner Munzel statistic
 
    Missing values in `x` and/or `y` are discarded.
 
    Parameters
    ----------
    x : sequence
        Input
    y : sequence
        Input
    alternative : {greater, less, two_sided }
 
    Returns
    -------
    statistic : float
        The Brunner Munzel  statistics
    pvalue : float
        Approximate p-value assuming a t distribution.
 
     """
    x = np.ma.asarray(x).compressed().view(np.ndarray)
    y = np.ma.asarray(y).compressed().view(np.ndarray)
    ranks = stats.rankdata(np.concatenate([x,y]))
    (nx, ny) = (len(x), len(y))
    rankx = stats.rankdata(x)
    ranky = stats.rankdata(y)
    rank_mean1 = np.mean(ranks[0:nx])
    rank_mean2 = np.mean(ranks[nx :nx+ny])
#     print(ranks)
#     print(ranks[0:nx],ranks[nx :nx+ny])
 
    pst = (rank_mean2 - (ny + 1)/2)/nx
     
    v1_set = [(i - j - rank_mean1 + (nx + 1)/2)**2 for (i,j) in zip(ranks[0:nx],rankx)]
    v2_set = [(i - j - rank_mean2 + (ny + 1)/2)**2 for (i,j) in zip(ranks[nx :nx+ny] ,ranky)]
 
    v1 = np.sum(v1_set)/(nx - 1)
    v2 = np.sum(v2_set)/(ny - 1)
    statistic = nx * ny * (rank_mean2 - rank_mean1)/(nx + ny)/np.sqrt(nx * v1 +  ny * v2)
    dfbm = ((nx * v1 + ny * v2)**2)/(((nx * v1)**2)/(nx - 1) + ((ny * v2)**2)/(ny - 1))
    if ((alternative == "greater") | (alternative == "g")) :
        prob = stats.t.cdf(statistic,dfbm)
    elif ((alternative == "less") | (alternative == "l")) :
        prob = 1-stats.t.cdf(statistic,dfbm)
    else:
        alternative = "two_sided"
        abst = np.abs(statistic)
        prob  = stats.t.cdf(abst,dfbm)
        prob= 2 * min(prob, 1-prob)
     
    return BrunnerMunzelResult(statistic,prob)

In [29]:
statistic, p_value = brunner_munzel_test(sample1, sample2)
print(statistic)
print(p_value)

-18.99344323580644
0.0
